# 실습 5: Tools 생성

## 목표
- Disaster Tools 구현 (소방서 검색, 기상 정보, 위키피디아)
- Browser Tool 구현 (뉴스 검색)

## 5-1. 설정 확인

In [ ]:
from workshop_config import REGION, WINDY_API_KEY

print(f"🌍 Region: {REGION}")
print(f"🌤️ Windy API Key: {'설정됨' if WINDY_API_KEY else '⚠️ 미설정'}")

## 5-2. Disaster Tools 생성

Strands Agent에서 사용할 Tool들을 정의합니다:
- `@tool` 데코레이터로 함수를 Tool로 등록
- docstring이 Tool 설명으로 사용됨

In [ ]:
%%writefile agent/deploy/runtime/tool_use/disaster_tools.py
"""화재 대응 도구 모음"""

import wikipedia as wiki
import requests
from strands import tool
import boto3
import json
import math

from workshop_config import (
    REGION, WINDY_API_KEY, WINDY_API_URL,
    VECTOR_BUCKET_NAME, VECTOR_INDEX_NAME
)

@tool
def search_wikipedia(query: str) -> str:
    """위키피디아에서 정보를 검색합니다.
    
    Args:
        query: 검색할 키워드
    
    Returns:
        검색 결과 요약
    """
    try:
        wiki.set_lang("ko")
        page = wiki.page(query)
        summary = page.summary[:500] + "..." if len(page.summary) > 500 else page.summary
        return f"제목: {page.title}\n\n{summary}\n\n출처: {page.url}"
    except Exception as e:
        return f"검색 실패: {str(e)}"

@tool
def find_fire_station(address: str) -> str:
    """주소 기반으로 가까운 소방서 5곳을 벡터 검색으로 찾습니다.
    
    Args:
        address: 화재 발생 주소
    
    Returns:
        가까운 소방서 5곳 정보
    """
    try:
        bedrock = boto3.client("bedrock-runtime", region_name=REGION)
        s3vectors = boto3.client("s3vectors", region_name=REGION)
        
        # 주소 임베딩
        resp = bedrock.invoke_model(
            modelId="amazon.titan-embed-text-v2:0",
            body=json.dumps({"inputText": address, "dimensions": 1024, "normalize": True})
        )
        query_embedding = json.loads(resp["body"].read())["embedding"]
        
        # 벡터 검색
        results = s3vectors.query_vectors(
            vectorBucketName=VECTOR_BUCKET_NAME,
            indexName=VECTOR_INDEX_NAME,
            queryVector={"float32": query_embedding},
            topK=5,
            returnMetadata=True
        )
        
        if results.get("vectors"):
            output = f"화재 발생 주소: {address}\n\n가까운 소방서 5곳:\n\n"
            for i, item in enumerate(results["vectors"], 1):
                m = item.get("metadata", {})
                output += f"{i}. {m.get('name')}\n"
                output += f"   - 위도: {m.get('latitude')}, 경도: {m.get('longitude')}\n"
                output += f"   - 주소: {m.get('address')}\n"
                output += f"   - 전화: {m.get('phone')}\n\n"
            return output
        return "소방서를 찾을 수 없습니다."
    except Exception as e:
        return f"오류: {str(e)}"

@tool
def get_weather_info(latitude: float, longitude: float) -> str:
    """Windy API로 기상 정보를 조회합니다.
    
    Args:
        latitude: 위도
        longitude: 경도
    
    Returns:
        기상 정보 (온도, 풍속, 풍향, 습도, 기압)
    """
    try:
        payload = {
            "lat": latitude, "lon": longitude,
            "model": "gfs",
            "parameters": ["wind", "temp", "rh", "pressure"],
            "levels": ["surface"],
            "key": WINDY_API_KEY
        }
        
        resp = requests.post(WINDY_API_URL, json=payload, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        
        if data and data.get("ts"):
            wind_u = data.get("wind_u-surface", [0])[0]
            wind_v = data.get("wind_v-surface", [0])[0]
            temp_k = data.get("temp-surface", [273.15])[0]
            humidity = data.get("rh-surface", [0])[0]
            pressure = data.get("pressure-surface", [1013])[0]
            
            wind_speed = math.sqrt(wind_u**2 + wind_v**2)
            wind_dir = (math.atan2(wind_u, wind_v) * 180 / math.pi + 180) % 360
            temp_c = temp_k - 273.15
            
            return (f"위도: {latitude}, 경도: {longitude}\n"
                    f"온도: {temp_c:.1f}°C\n풍속: {wind_speed:.1f} m/s\n"
                    f"풍향: {wind_dir:.0f}°\n습도: {humidity}%\n기압: {pressure} hPa")
        return "기상 정보를 가져올 수 없습니다."
    except Exception as e:
        return f"기상 정보 조회 오류: {str(e)}"

## 5-3. Browser Tool 생성

In [ ]:
%%writefile agent/deploy/runtime/tool_use/browser_tool/playwright.py
"""Browser tool using Playwright

strands_tools의 browser_tool을 사용합니다.
뉴스 검색 및 웹 페이지 정보 수집에 활용됩니다.
"""

from strands_tools import browser_tool

## 5-4. workshop_config.py 동기화

In [ ]:
import shutil
shutil.copy('workshop_config.py', 'agent/deploy/runtime/')
print("✅ workshop_config.py 동기화 완료")

## ✅ 실습 5 완료!

### 생성된 파일
- `agent/deploy/runtime/tool_use/disaster_tools.py`
  - `search_wikipedia`: 위키피디아 검색
  - `find_fire_station`: S3 Vectors 기반 소방서 검색
  - `get_weather_info`: Windy API 기상 정보
- `agent/deploy/runtime/tool_use/browser_tool/playwright.py`

👉 다음: **lab6_agent_runtime.ipynb**